In [1]:
import sys
sys.path.append('../../build')
import numpy as np
import libry as ry
import time
import cv2 as cv

**ry-c++-log** ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback

**ry-c++-log** util.cpp:initCmdLine:545(1) ** cmd line arguments: 'rai-pybind -python '** INFO:ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback


**ry-c++-log** util.cpp:initCmdLine:549(1) ** run path: '/home/vasko/Documents/TUB3/AI_Robotics/robotics-course/course3-Simulation/05-grasp'
** INFO:util.cpp:initCmdLine:545(1) ** cmd line arguments: 'rai-pybind -python '

**ry-c++-log** graph.cpp:initParameters:1379(1) ** parsed parameters:
{python}

** INFO:util.cpp:initCmdLine:549(1) ** run path: '/home/vasko/Documents/TUB3/AI_Robotics/robotics-course/course3-Simulation/05-grasp'

** INFO:graph.cpp:initParameters:1379(1) ** parsed parameters:
{python}



In [2]:
#Let's edit the real world before we create the simulation
RealWorld = ry.Config()
RealWorld.addFile("../../scenarios/challenge.g")

#you can also change the shape & size
RealWorld.getFrame("obj0").setColor([1.,0,0])
RealWorld.getFrame("obj0").setShape(ry.ST.sphere, [.03])
# RealWorld.getFrame("obj0").setPosition([0., .2, 1.])
RealWorld.getFrame("obj0").setPosition([0., .2, 2.])

RealWorld.getFrame("obj0").setContact(1)

#remove some objects
for o in range(1,30):
    name = "obj%i" % o
#     print("deleting", name)
    RealWorld.delFrame(name)
    
    
# instantiate the simulation
S = RealWorld.simulation(ry.SimulatorEngine.bullet, True)
S.addSensor("camera")

# we're adding an "imp" to the simulation, which is a little process that can inject perturbations
S.addImp(ry.ImpType.objectImpulses, ['obj0'], [])

In [3]:
# setup your model world
C = ry.Config()
C.addFile("../../scenarios/pandasTable.g")

obj = C.addFrame("object")
obj.setColor([1.,0,0]) # set the color of one objet to red!
obj.setShape(ry.ST.sphere, [.03]) # assume that we know the object shape

# obj.setQuaternion([1,0,1,0])
# obj.setShape(ry.ST.capsule, [.2,.02])
# obj.setColor([1,0,1])

D = C.view();

In [4]:
# camera parameters
cameraFrame = RealWorld.frame("camera")
camInfo = cameraFrame.info()

f = camInfo['focalLength']; W=camInfo['width']; H=camInfo['height']; #from the scenario file
                
fx, fy = f*H, f*H
px, py = W/2, H/2
R, off = cameraFrame.getRotationMatrix(), cameraFrame.getPosition()
R,off


(array([[ 1.       ,  0.       ,  0.       ],
        [ 0.       ,  0.8660254, -0.5      ],
        [ 0.       ,  0.5      ,  0.8660254]]),
 array([-0.01, -0.2 ,  1.8 ]))

In [5]:
def red_obj_det(image):
    result = image.copy()

    image_hsv = cv.cvtColor(image, cv.COLOR_RGB2HSV)

    # lower boundary RED color range values; Hue (0 - 10)
    lower1 = np.array([0, 100, 20])
    upper1 = np.array([10, 255, 255])

    # upper boundary RED color range values; Hue (160 - 180)
    lower2 = np.array([160,100,20])
    upper2 = np.array([179,255,255])

    lower_mask = cv.inRange(image_hsv, lower1, upper1)
    upper_mask = cv.inRange(image_hsv, lower2, upper2)

    full_mask = lower_mask + upper_mask;

    result = cv.bitwise_and(result, result, mask=full_mask)

    mask_greyscale = np.repeat(full_mask[:,:,np.newaxis],3,axis=2)
    
    contours, hierarchy = cv.findContours(cv.cvtColor(mask_greyscale, cv.COLOR_RGB2GRAY),cv.RETR_TREE,cv.CHAIN_APPROX_NONE)
    
    return contours, full_mask


def pos_image_to_world(cameraFrame, x, y, Z):
    ## depth is sign-fliped, j: right, i: down
    ## Camera Frame
    pos = np.zeros(3)
    pos[0] = Z * (x - px) / fx;
    pos[1] = -Z * (y - py) / fy;
    pos[2] = -Z
    
    ## Coordinate transformation (from camera to world) 
    pos = (R@pos[:,np.newaxis]).flatten() + off
    
    return pos

def z_from_contour(depth, contour):
    global o_depth, o_contour

    o_depth = depth
    o_contour = contour
    c = contour.reshape(-1,2)
    return depth[c[:,1], c[:,0]].mean()

def estimateBallPosition(depth, contour):
    Z_mean = z_from_contour(depth, contour)
    
    M = cv.moments(contour)
    
    if (M["m00"]):
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
    
        pos = pos_image_to_world(cameraFrame, cX, cY, Z_mean)
    else:
        pos = np.zeros(3)
        print("Error: Ball not detected!")
    
    return pos 

In [6]:
def find_moving_ball(prev_contours, prev_full_mask, contours, full_mask):       
    solution = contours[0]

    if prev_contours != None:
        diff_mask = (full_mask - prev_full_mask).clip(0,255)

        for c in contours:
            image_mask = cv.drawContours(np.zeros(full_mask.shape, np.uint8), [c], -1, 255, cv.FILLED)
            combination_mask = cv.bitwise_and(diff_mask, image_mask)
            if (np.count_nonzero(combination_mask)>20):
                solution = c
                break;
    return solution

In [7]:
def solve_komo_path_grasping(waypoints = 5, horizon_seconds =.5):
    komo = C.komo_path(1., waypoints, horizon_seconds, False)

    # encode grasping
    komo.addObjective([1.], 
                    ry.FS.positionRel, 
                    ["object", "R_gripperCenter"], 
                    ry.OT.eq, 
                    1e2*np.array([[1,0,0],[0,1,0]]),
                    [0.,0.,.0]);

    # encode hight tracking
    komo.addObjective([0.,.5], 
                ry.FS.positionDiff, 
                ["object", "R_gripperCenter"], 
                ry.OT.eq, 
                1.2e2*np.array([[0,0,1]]),
                [0.,0.,.0]);
    
    # add the pre-positon (TO BE ABOVE THE BALL and then go to the ball at the end)
    komo.addObjective([.8], 
                ry.FS.positionDiff, 
                ["object", "R_gripperCenter"], 
                ry.OT.eq, 
                1.2e2*np.array([[0,0,1]]),
                [0.,0.,.0]);
    
    
    komo.addObjective([0.,1.], 
                    ry.FS.quaternionRel, 
                    ["world", "R_gripperCenter"], 
                    ry.OT.eq, 
                    [1e2],
                    [.579,.406,-.406,.579]);

    komo.addObjective([1.], ry.FS.qItself, [], ry.OT.eq, [1e2], order=1)

    komo.optimize()
#     komo.getReport()
    return komo.getPathFrames() 

def solve_komo_path_lifting(release_pos = [0., 0., 1.5], waypoints = 5, horizon_seconds =.5):
    komo = C.komo_path(1., waypoints, horizon_seconds, False)

    komo.addObjective([1.], 
                    ry.FS.positionRel, 
                    ["R_gripperCenter", "world"], 
                    ry.OT.eq, 
                    1e2,
                    release_pos);


    komo.addObjective([0.,1.], 
                    ry.FS.quaternionRel, 
                    ["world", "R_gripperCenter"], 
                    ry.OT.eq, 
                    [1e2],
                    [.579,.406,-.406,.579]);


    komo.addObjective([1.], ry.FS.qItself, [], ry.OT.eq, [1e2], order=1)

    komo.optimize()
#     komo.getReport()
    return komo.getPathFrames() 

In [8]:
def solve_komo_IK_grasping():
    IK = C.komo_IK(False)

    # encode grasping
    IK.addObjective([1.], 
                    ry.FS.positionRel, 
                    ["object", "R_gripperCenter"], 
                    ry.OT.eq, 
                    1e2*np.array([[1,0,0],[0,1,0]]),
                    [0.,0.,.0]);

    # encode hight tracking
    IK.addObjective([0.,.5], 
                ry.FS.positionDiff, 
                ["object", "R_gripperCenter"], 
                ry.OT.eq, 
                1.2e2*np.array([[0,0,1]]),
                [0.,0.,.0]);

    IK.addObjective([0.,1.], 
                    ry.FS.quaternionRel, 
                    ["world", "R_gripperCenter"], 
                    ry.OT.eq, 
                    [1e2],
                    [.579,.406,-.406,.579]);


    IK.optimize()
#     IK.getReport()
    return IK.getPathFrames()  

def solve_komo_IK_lifting(release_pos = [0., 0., 1.5]):
    IK = C.komo_IK(False)

    IK.addObjective([1.], 
                    ry.FS.positionRel, 
                    ["R_gripperCenter", "world"], 
                    ry.OT.eq, 
                    1e2,
                    release_pos);


    IK.addObjective([0.,1.], 
                    ry.FS.quaternion, 
                    ["world", "R_gripperCenter"], 
                    ry.OT.eq, 
                    [1e2],
                    [.579,.406,-.406,.579]);


    IK.optimize()
#     IK.getReport()
    return IK.getPathFrames() 

In [9]:
def grasping(KOMO = False):
    points = []
    tau = .01

    gripping = False
    grasped = False

    prev_contours = None
    prev_full_mask = None
    # small cheating, better intitialization
    objectPosition = RealWorld.frame("obj0").getPosition();

    for t in range(3000):
        time.sleep(tau)

        #grab sensor readings from the simulation
        q = S.get_q()
        if t%10 == 0:
            [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow

            contours, full_mask = red_obj_det(rgb)
            if (len(contours)!=0):
                mv_ball_contour = find_moving_ball(prev_contours, prev_full_mask, contours, full_mask)

                objectPosition = estimateBallPosition(depth, mv_ball_contour)
                prev_contours, prev_full_mask = contours, full_mask

        # TOTAL CHEAT: grab the true position from the RealWorld
    #     objectPosition = RealWorld.frame("obj0").getPosition();
        obj.setPosition(objectPosition) # set the model object to percept

        C.setJointState(q); # set your robot model to match the real q

        y, J = C.evalFeature(ry.FS.positionDiff, ["R_gripperCenter", "object"])

    #     qdot = np.linalg.inv(J.T@J + 1e-3*np.eye(q.shape[0])) @ J.T @ (-5*y)

#         komo_next_state = solve_komo_IK_grasping()[0]
        if KOMO:
            if t%10 == 0:
                komo_next_state = solve_komo_path_grasping()[0]
        else:
            komo_next_state = solve_komo_IK_grasping()[0]
            
        C.setFrameState(komo_next_state)

        qdot = ( C.getJointState() - q ) /  (  40*tau )

        if not gripping and np.linalg.norm(y) < .01:
            S.closeGripper("R_gripper", speed=1.)
            gripping = True

        if gripping and S.getGripperWidth("R_gripper") < 1e-2:
            print("FAILED!")
            S.openGripper("R_gripper", speed=1.)
            gripping = False
            return False

        if gripping and S.getGripperIsGrasping("R_gripper"):
            print("GRASPED!")
            grasped = True
            return True

        S.step(qdot, tau, ry.ControlMode.velocity)
    
    

In [10]:
def lift_release(release_pos = [0., 0., 1.5], released = False, KOMO = False):
    
    points = []
    tau = .01
    
    releasing = False

    for t in range(3000):
        time.sleep(tau)

        #grab sensor readings from the simulation
        q = S.get_q()

        C.setJointState(q); # set your robot model to match the real q

        y, J = C.evalFeature(ry.FS.positionRel, ["R_gripperCenter", "world"])

    #     qdot = np.linalg.inv(J.T@J + 1e-3*np.eye(q.shape[0])) @ J.T @ (-5*y)

        if KOMO:
            if t%10 == 0:
                komo_next_state = solve_komo_path_lifting(release_pos)[0]
        else:
            komo_next_state = solve_komo_IK_lifting(release_pos)[0]
            
        C.setFrameState(komo_next_state)

        qdot = ( C.getJointState() - q ) /  (  40*tau )

        if not released and np.linalg.norm(y-release_pos) < .01:
            S.openGripper("R_gripper", speed=1.)
            releasing = True
        
        if releasing and S.getGripperWidth("R_gripper") > .03:
            released = True
            releasing = False
            return True

        S.step(qdot, tau, ry.ControlMode.velocity)
    
    return False
    

In [11]:
def lift_trow(trow_pos = [0., 0., 1.2], waypoints = 5, horizon_seconds =.6):
    
    points = []
    tau = .01
    komo_step = 0
    released = False
    releasing = False
    komo_done = False

    for t in range(3000):
        time.sleep(tau)

        #grab sensor readings from the simulation
        q = S.get_q()

        C.setJointState(q); # set your robot model to match the real q

        y, J = C.evalFeature(ry.FS.positionRel, ["R_gripperCenter", "world"])

    #     qdot = np.linalg.inv(J.T@J + 1e-3*np.eye(q.shape[0])) @ J.T @ (-5*y)

        if ((t*tau)%(horizon_seconds/(waypoints+1)) < 1.e-9) and not komo_done:
            if komo_step == 0:
                
                komo_next_states = solve_komo_path_lifting(2*np.array(trow_pos))

                C.setFrameState(komo_next_states[komo_step])
                komo_step+=1
            else:
                print("komo_step {}".format(komo_step))
                C.setFrameState(komo_next_states[komo_step])
                komo_step+=1
                if (komo_step == waypoints):
                    komo_step = 0
                    komo_done = True
            
            print("Speed calculation")
            qdot = ( C.getJointState() - q ) /  (  40*tau )
        elif t*tau > (horizon_seconds+.1):
            qdot = .1*qdot

        if not released and np.abs(y[2]-trow_pos[2]) < .1:
            print(y)
            print(trow_pos)
            S.openGripper("R_gripper", speed=1.)
            releasing = True
        
        if releasing and S.getGripperWidth("R_gripper") > .03:
            released = True
            releasing = False

        S.step(qdot, tau, ry.ControlMode.velocity)
    
    return released
    

In [12]:
# while (True):
#     grasped = grasping(True)
#     if not grasped:
#         print("Failiure grasping")
#         break;
#     released = lift_release()
#     if not released:
#         print("Failiure releasing")
#         break;

In [13]:
while (True):
    grasped = grasping()
    if not grasped:
        print("Failiure grasping")
        break;
    released = lift_trow()
    if not released:
        print("Failiure releasing")
        break;

** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.063273 (kin:0.004606 coll:0 feat:0.039155 newton: 0.003654) setJointStateCount:1001
   sos:0.101143 ineq:0 eq:98.1499
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.063874 (kin:0.002961 coll:0 feat:0.034679 newton: 0.004677) setJointStateCount:1001
   sos:0.097102 ineq:0 eq:97.977
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.058093 (kin:0.005184 coll:0 feat:0.034225 newton: 0.003448) setJointStateCount:1001
   sos:0.0920621 ineq:0 eq:98.1682
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.047052 (kin:0.004144 coll:0 feat:0.022072 

** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.052634 (kin:0.006781 coll:0 feat:0.024867 newton: 0.004806) setJointStateCount:1001
   sos:0.0172344 ineq:0 eq:98.2653
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.053037 (kin:0.00374 coll:0 feat:0.028302 newton: 0.003338) setJointStateCount:1000
   sos:0.0171467 ineq:0 eq:97.6844
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.05673 (kin:0.002745 coll:0 feat:0.030861 newton: 0.005187) setJointStateCount:1001
   sos:0.0162476 ineq:0 eq:97.5912
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.057311 (kin:0.002938 coll:0 feat:0.027218

** optimization time:0.000525 (kin:2e-05 coll:0 feat:0.000172 newton: 4.4e-05) setJointStateCount:8
   sos:0.0129094 ineq:0 eq:0.0157005
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000487 (kin:2.1e-05 coll:0 feat:0.00019 newton: 4.4e-05) setJointStateCount:8
   sos:0.012235 ineq:0 eq:0.015421
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000447 (kin:2.3e-05 coll:0 feat:0.000168 newton: -6.83056) setJointStateCount:7
   sos:0.0116045 ineq:0 eq:0.0501497
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000815 (kin:3.8e-05 coll:0 feat:0.000343 newton: -6.86745) setJointStateCount:6
   sos:0.00649552 ineq:0 eq:0.0439903
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases

** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000366 (kin:1.4e-05 coll:0 feat:0.000126 newton: -7.52334) setJointStateCount:5
   sos:0.00384013 ineq:0 eq:0.0157375
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.00107 (kin:4.8e-05 coll:0 feat:0.000398 newton: -7.56351) setJointStateCount:5
   sos:0.00362264 ineq:0 eq:0.0275955
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000516 (kin:2.1e-05 coll:0 feat:0.000177 newton: -15.2317) setJointStateCount:4
   sos:0.00345776 ineq:0 eq:0.0726231
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000705 (kin:2.7e-05 coll:0 feat:0.000258 ne

** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.001065 (kin:4.8e-05 coll:0 feat:0.000425 newton: -8.64875) setJointStateCount:5
   sos:0.0040473 ineq:0 eq:0.0177733
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000613 (kin:2.4e-05 coll:0 feat:0.000202 newton: -17.3906) setJointStateCount:4
   sos:0.00271361 ineq:0 eq:0.0305185
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000371 (kin:1.3e-05 coll:0 feat:0.000121 newton: -8.72202) setJointStateCount:4
   sos:0.00257117 ineq:0 eq:0.0287277
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000277 (kin:1.3e-05 coll:0 feat:0.0001 newt

...

** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000289 (kin:8e-06 coll:0 feat:8.7e-05 newton: -10.1991) setJointStateCount:3
   sos:0.000127449 ineq:0 eq:0.0174118
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000752 (kin:3.7e-05 coll:0 feat:9.2e-05 newton: -10.2081) setJointStateCount:3
   sos:0.000126769 ineq:0 eq:0.0177131
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000258 (kin:1e-05 coll:0 feat:8.5e-05 newton: -10.2316) setJointStateCount:3
   sos:0.000120069 ineq:0 eq:0.0162905
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000857 (kin:3.1e-05 coll:0 feat:0.000282 newto

** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000467 (kin:1.6e-05 coll:0 feat:0.000127 newton: -10.8943) setJointStateCount:3
   sos:0.000263856 ineq:0 eq:0.017732
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000303 (kin:1.3e-05 coll:0 feat:0.000102 newton: -10.945) setJointStateCount:3
   sos:0.000259917 ineq:0 eq:0.0172605
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000577 (kin:1.8e-05 coll:0 feat:0.000172 newton: -10.9695) setJointStateCount:3
   sos:0.000240834 ineq:0 eq:0.0171564
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000811 (kin:2.9e-05 coll:0 feat:0.000263 

** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000299 (kin:1.2e-05 coll:0 feat:0.000113 newton: -11.8125) setJointStateCount:3
   sos:4.85483e-05 ineq:0 eq:0.0159228
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000256 (kin:9e-06 coll:0 feat:7.6e-05 newton: -11.8363) setJointStateCount:3
   sos:4.77861e-05 ineq:0 eq:0.0269309
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000266 (kin:1.2e-05 coll:0 feat:7.7e-05 newton: -11.8439) setJointStateCount:3
   sos:4.8346e-05 ineq:0 eq:0.0163389
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000445 (kin:1.5e-05 coll:0 feat:0.000141 new

** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000441 (kin:1.9e-05 coll:0 feat:0.000177 newton: -12.7724) setJointStateCount:3
   sos:5.02048e-05 ineq:0 eq:0.0166188
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000949 (kin:3.5e-05 coll:0 feat:0.000346 newton: -12.799) setJointStateCount:3
   sos:4.53915e-05 ineq:0 eq:0.0366215
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000381 (kin:1.4e-05 coll:0 feat:0.000118 newton: -12.8567) setJointStateCount:3
   sos:4.35819e-05 ineq:0 eq:0.0154279
** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000247 (kin:8e-06 coll:0 feat:8.4e-05 ne

KeyboardInterrupt: 

In [ ]:
# lift_release([0,0,.7],False)

In [ ]:
lift_trow()